In [1]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 1352



### data

In [2]:
sas.submit(f"""
    data prdsale;
        set sashelp.prdsale;
        if actual = 608 then country = '';
        if actual = 642 then do;
           actual = .;
           predict = .;
        end;
    run;
           
    data df1;
        set prdsale;
        where actual = 925;
    run;
              
    data df2;
        set prdsale;
        where actual = 999;
    run;
""")
sd1 = sas.sasdata("df1")
sd2 = sas.sasdata("df2")

In [3]:
sd = sas.sasdata("prdsale","work")
sd.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01
2,608.0,846.0,NaN,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-03-01
3,NaN,NaN,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-04-01
4,656.0,646.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-05-01


In [4]:
sd1.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,925.0,345.0,CANADA,WEST,CONSUMER,OFFICE,TABLE,4.0,1994.0,1994-12-01


In [5]:
sd2.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01


### set statement


In [6]:
# Interleave observations by a var;
var = "predict"
sas.submitLST(f"""
    proc sort data = df1; by {var}; run;
    proc sort data = df2; by {var}; run;

    DATA interleave; 
        SET df1 df2; 
        BY {var}; 
    RUN;
              
    PROC PRINT DATA = interleave;
    RUN;
""")

Obs,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
1,$999.00,$297.00,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1,1993,Feb
2,$925.00,$345.00,CANADA,WEST,CONSUMER,OFFICE,TABLE,4,1994,Dec
3,$925.00,$850.00,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1,1993,Jan


### match merge 


In [7]:
# one to one
sas.submitLST(f"""
    data _a;
        set interleave;
        keep month actual;
    run;
              
    data _b;
        set interleave;
        keep month predict;
    run;
    
    proc sort data = _a; by month; run;
    proc sort data = _b; by month; run;

    data _c;
        merge _a _b;
        by month;
    run;

    proc print data = _c;
    run;
""")

Obs,ACTUAL,MONTH,PREDICT
1,$925.00,Jan,$850.00
2,$999.00,Feb,$297.00
3,$925.00,Dec,$345.00


In [8]:
# one to many
sas.submitLST(f"""
proc sort data = prdsale out = _sorted; by country; run;

proc means data = _sorted noprint;
    by country;
    var predict;
    output out = _mean mean = mean;
run;

proc sort data = _mean; by country; run;

data _df;
    merge _sorted _mean;
    by country;
    keep actual predict country mean;
run;

proc sort data = _df; by actual; run;

proc print data = _df (obs = 5);
run;

""")

Obs,ACTUAL,PREDICT,COUNTRY,mean
1,.,.,CANADA,$483.73
2,$3.00,$405.00,CANADA,$483.73
3,$3.00,$938.00,GERMANY,$482.40
4,$4.00,$975.00,U.S.A.,$503.48
5,$5.00,$425.00,CANADA,$483.73


In [9]:
print(f"{sas.sasdata("prdsale","sashelp").obs()=}")
print(f"{sas.sasdata("_df","work").obs()=}")

sas.sasdata("prdsale","sashelp").obs()=1440
sas.sasdata("_df","work").obs()=1440


### grand total


In [10]:
# use the if _n_ = 1 then set ; trick
sas.submitLST(f"""
proc means data = prdsale noprint;
    var predict;
    output out = _mean mean = m_predict;
run;
              
data _df;
    set prdsale;
    if _n_ = 1 then set _mean;
    keep actual predict country m_predict;
run;
              
proc print data = _df (obs = 5);
run;
""")

Obs,ACTUAL,PREDICT,COUNTRY,m_predict
1,$925.00,$850.00,CANADA,$490.45
2,$999.00,$297.00,CANADA,$490.45
3,$608.00,$846.00,,$490.45
4,.,.,CANADA,$490.45
5,$656.00,$646.00,CANADA,$490.45


### output statement to unstack the data


In [11]:
sas.sasdata("prdsale","work").top("country")

,COUNTRY,COUNT,PERCENT
0,NaN,3.0,NaN
1,GERMANY,480.0,33.402923
2,U.S.A.,479.0,33.333333
3,CANADA,478.0,33.263744


In [12]:
sas.submitLST(f"""
data ger usa can other;
    set prdsale;
    if country = 'GERMANY' then output ger;
    else if country = 'U.S.A.' then output usa;
    else if country = 'CANADA' then output can;
    else output other;
run;
""")



101                                                        The SAS System                          Tuesday, May  7, 2024 01:13:00 PM

694        ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
694      !  graphics on / outputfmt=png;
695        
696        
697        data ger usa can other;
698            set prdsale;
699            if country = 'GERMANY' then output ger;
700            else if country = 'U.S.A.' then output usa;
701            else if country = 'CANADA' then output can;
702            else output other;
703        run;
704        
705        
706        
707        ods html5 (id=saspy_internal) close;ods listing;
708        

102                                                        The SAS System                          Tuesday, May  7, 2024 01:13:00 PM

709        


In [13]:
print(f"{sas.sasdata('ger').obs()=}")
print(f"{sas.sasdata('usa').obs()=}")
print(f"{sas.sasdata('can').obs()=}")
print(f"{sas.sasdata('other').obs()=}")

sas.sasdata('ger').obs()=480
sas.sasdata('usa').obs()=479
sas.sasdata('can').obs()=478
sas.sasdata('other').obs()=3


### ouput to generate data

In [14]:
# no input or set so only 1 iteration
sas.submitLST(f"""
data _df;
    do x = 1 to 100;
        y = x**2;
        output;
    end;
run;
              
proc print data = _df (obs = 5);
run;
""")

Obs,x,y
1,1,1
2,2,4
3,3,9
4,4,16
5,5,25
